In [1]:
# Qpt

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_base import ProjectedGradientDescentBase, ProjectedGradientDescentBaseOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement,
    get_y_measurement,
    get_z_measurement
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [6]:
tester_povms = [get_x_measurement(c_sys), get_y_measurement(c_sys), get_z_measurement(c_sys)]

In [7]:
# True Object
true_objects = []
#true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
#true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
#true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

true_object = true_objects[0]

In [8]:
num_data = [100, 1000, 10000]
n_rep = 10

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(12),
    WeightedRelativeEntropy(16),
    WeightedProbabilityBasedSquaredError(12),
    WeightedProbabilityBasedSquaredError(16),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption(),
    WeightedRelativeEntropyOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       loss=loss,
       loss_option=loss_option,
       algo=algo,
       algo_option=algo_option,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/10 [00:00<?, ?it/s]Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.040246419111887616[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.04073441028594971[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.06119064092636108[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.061413562297821044[min]

Case 4: Maximum-Likelihood(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LossMinimizationEstimator
  0%|          | 0/10 [00:00<?, ?i

AttributeError: 'NoneType' object has no attribute 'set_func_prob_dists_from_standard_qt'

In [9]:
fig = data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_name_list,
    true_object,
)
fig.show()

In [11]:
report.export_report(
    "qpt_nrep=10.pdf",
    estimation_results_list,
    case_name_list,
    estimator_list,
    true_object,
    tester_states + tester_povms,
    loss_list=loss_list,
    loss_option_list=loss_option_list,
    algo_list=algo_list,
    algo_option_list=algo_option_list,
    seed=seed,
    computation_time=sum(elapsed_times)
)

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...


ValueError: arrays must all be same length

In [13]:
path = "output/qpt_estimation_results_nrep=1000_all.pickle"
with open(path, 'rb') as f:
    all_results = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'output/qpt_estimation_results_nrep=1000_all.pickle'

In [12]:
true_object_names = ["depolarizing_channel_p=0",
"depolarizing_channel_p=0.05",
"depolarizing_channel_p=1",
"x_rotation_theta=np.pi_half",
"x_rotation_theta=np.pi",
"amplitutde_damping_channel_gamma=0.1"]

dir_name = Path("output_qpt_nrep=1000_re_report")
for i, result in enumerate(all_results):
    print("==========================")
    print(f"{i}: {true_object_names[i]}")
    print("==========================")

    name = true_object_names[i]
    report.export_report(
    dir_name / f"qpt_nrep=1000_case={i}_{name}.pdf",
    result["estimation_results_list"],
    case_name_list,
    estimator_list,
    result["true_object"],
    tester_states + tester_povms,
    seed=seed,
    computation_time=sum(result["elapsed_times"])
    )

NameError: name 'all_results' is not defined